In [1]:
from dataloader import DataLoader, DataTransformer
from model import Model
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import backtrader as bt
%matplotlib inline

Using TensorFlow backend.


In [20]:
tickers = ['SPY']
from_date = datetime(2000, 1, 1)
to_date = datetime(2016, 12, 31)
entry_percentage = 0.65

In [3]:
loader = DataLoader(tickers, from_date=from_date, to_date=to_date)
loader.reload_all()

In [4]:
transformer = DataTransformer(loader)
transformer.transform()

In [5]:
model = Model(transformer)
model.build_model_data()
model.build_neural_net()

In [6]:
datetime(2016, 12, 31).strftime('%Y-%m-%d')

'2016-12-31'

In [15]:
model.predict_one('SPY', '2010-10-18')[0]

array([ 0.55760902,  0.44239092], dtype=float32)

In [26]:
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, %.2f' % order.executed.price)
            elif order.issell():
                self.log('SELL EXECUTED, %.2f' % order.executed.price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        
        self.prev_order = self.order
        self.order = None

    def next(self):
        #self.log('Close, %.2f' % self.dataclose[0])
        
        if self.order:
            return
        
        if not self.position:
            try:
                prediction = model.predict_one('SPY', self.datas[0].datetime.date(0).isoformat())[0]
                if prediction[0] > entry_percentage:
                    self.order = self.buy()
                elif prediction[1] > entry_percentage:
                    self.order = self.sell()
            except Exception as e:
                self.log(e)
        else:
            if len(self) >= (self.bar_executed + 5):
                if self.prev_order.isbuy():
                    self.order = self.sell()
                else:
                    self.order = self.buy()

In [29]:
cerebro = bt.Cerebro()
cerebro.addstrategy(TestStrategy)

data = bt.feeds.YahooFinanceData(
    dataname='SPY',
    fromdate=datetime(2000, 1, 1),
    todate=datetime(2016, 12, 31),
    reverse=False)

cerebro.adddata(data)
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addsizer(bt.sizers.PercentSizer, percents=3)

cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')

print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
results = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

strat = results[0]
pyfoliozer = strat.analyzers.getbyname('pyfolio')
returns, positions, transactions, gross_lev = pyfoliozer.get_pf_items()

Starting Portfolio Value: 100000.00
2000-01-03, 'the label [2000-01-03] is not in the [index]'
2000-01-04, 'the label [2000-01-04] is not in the [index]'
2000-01-05, 'the label [2000-01-05] is not in the [index]'
2000-01-06, 'the label [2000-01-06] is not in the [index]'
2000-01-07, 'the label [2000-01-07] is not in the [index]'
2000-01-10, 'the label [2000-01-10] is not in the [index]'
2000-01-11, 'the label [2000-01-11] is not in the [index]'
2000-01-12, 'the label [2000-01-12] is not in the [index]'
2000-01-13, 'the label [2000-01-13] is not in the [index]'
2000-01-14, 'the label [2000-01-14] is not in the [index]'
2000-01-18, 'the label [2000-01-18] is not in the [index]'
2000-01-19, 'the label [2000-01-19] is not in the [index]'
2000-01-20, 'the label [2000-01-20] is not in the [index]'
2000-01-21, 'the label [2000-01-21] is not in the [index]'
2000-01-24, 'the label [2000-01-24] is not in the [index]'
2000-01-25, 'the label [2000-01-25] is not in the [index]'
2000-01-26, 'the lab

2009-09-02, BUY EXECUTED, 99.78
2009-09-11, SELL EXECUTED, 104.99
2009-10-02, BUY EXECUTED, 102.02
2009-10-12, SELL EXECUTED, 107.76
2009-11-02, BUY EXECUTED, 104.13
2009-11-10, SELL EXECUTED, 109.31
2009-12-04, BUY EXECUTED, 111.84
2009-12-14, SELL EXECUTED, 111.87
2010-02-02, SELL EXECUTED, 109.26
2010-02-10, BUY EXECUTED, 107.05
2010-02-11, BUY EXECUTED, 106.87
2010-02-22, SELL EXECUTED, 111.55
2010-03-10, BUY EXECUTED, 114.51
2010-03-18, SELL EXECUTED, 117.11
2010-04-05, BUY EXECUTED, 118.25
2010-04-13, SELL EXECUTED, 119.62
2010-05-21, BUY EXECUTED, 105.91
2010-06-01, SELL EXECUTED, 108.35
2010-06-03, SELL EXECUTED, 110.65
2010-06-11, BUY EXECUTED, 108.19
2010-06-14, BUY EXECUTED, 110.52
2010-06-22, SELL EXECUTED, 111.41
2010-06-24, SELL EXECUTED, 108.69
2010-07-02, BUY EXECUTED, 103.11
2010-07-15, SELL EXECUTED, 109.61
2010-07-23, BUY EXECUTED, 109.24
2010-07-29, BUY EXECUTED, 111.52
2010-08-06, SELL EXECUTED, 111.74
2010-08-16, BUY EXECUTED, 107.57
2010-08-24, SELL EXECUTED, 105

In [ ]:
import pyfolio as pf
pf.create_full_tear_sheet(
    returns,
    positions=positions,
    transactions=transactions,
    live_start_date='2005-05-01',  # This date is sample specific
    round_trips=True)

c:\programdata\anaconda3\envs\trading3\lib\site-packages\empyrical\utils.py:393: UserWarning: Yahoo Finance read failed: mktime argument out of range, falling back to Google
  UserWarning)


Entire data start date: 2001-12-18
Entire data end date: 2016-12-30
In-sample months: 40
Out-of-sample months: 139


,All,In-sample,Out-of-sample
Annual return,0.7%,0.4%,0.8%
Cumulative returns,11.3%,1.3%,9.9%
Annual volatility,3.3%,5.2%,2.5%
Sharpe ratio,0.23,0.10,0.34
Calmar ratio,0.36,0.20,0.45
Stability,0.96,0.67,0.95
Max drawdown,-2.0%,-2.0%,-1.8%
Omega ratio,1.10,1.04,1.14
Sortino ratio,0.34,0.15,0.49
Skew,0.09,0.05,0.17


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,2.00,2002-07-12,2002-07-25,2002-11-18,92
1,1.82,2008-11-20,2008-12-15,2009-02-17,64
2,1.48,2008-04-14,2008-07-15,2008-09-29,121
3,1.47,2008-10-06,2008-10-10,2008-11-06,24
4,1.41,2008-03-04,2008-03-31,2008-04-09,27
